# Tryout bert embedding

In [2]:
import torch.nn as nn
import torch
# import os
from pathlib import Path

In [4]:
# combine both negative and pos in train and test to train
dataSet = {}
dataSet['train'] = []
dataSet['test'] = []
for p in Path('data/text_classification/train/pos/').glob('*.txt'): # 1 is pos and 0 is neg
    entry = p.read_text(encoding='utf8')
    dataSet['train'].append((entry, 1))
for p in Path('data/text_classification/train/neg/').glob('*.txt'): # 1 is pos and 0 is neg
    entry = p.read_text(encoding='utf8')
    dataSet['train'].append((entry, 0))

for p in Path('data/text_classification/test/pos/').glob('*.txt'): # 1 is pos and 0 is neg
    entry = p.read_text(encoding='utf8')
    dataSet['test'].append((entry, 1))
for p in Path('data/text_classification/test/neg/').glob('*.txt'): # 1 is pos and 0 is neg
    entry = p.read_text(encoding='utf8')
    dataSet['test'].append((entry, 0))


print(dataSet['train'][0])

('Greatly enjoyed this 1945 mystery thriller film about a young woman, Nina Foch,(Julia Ross) who is out of work and has fallen behind in her rent and is desperate to find work. Julia reads an ad in the local London newspaper looking for a secretary and rushes out to try and obtain this position. Julia obtains the position and is hired by a Mrs. Hughes, (Dame May Witty) who requires that she lives with her employer in her home and wants her to have no involvement with men friends and Julia tells them she has no family and is free to devote her entire time to this job. George Macready, (Ralph Hughes) is the son of Mrs. Hughes and has some very strange desires for playing around with knives. This was a low budget film and most of the scenes were close ups in order to avoid the expense of a background and costs for scenery. This strange family all live in a huge mansion off the Cornwall Coast of England and there is secret doors and plenty of suspense.', 1)


In [3]:
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import re
# https://stackoverflow.com/questions/9662346/python-code-to-remove-html-tags-from-a-string

In [4]:
# with bert we don't need to split the sentences. But do remove the html
cleanedTrainData = []
CLEANHTML = re.compile('<.*?>')

for entry in dataSet['train']:
    # ! Hope the data doesn't contain heavy html tags or else it wouldn't work
    text = re.sub(CLEANHTML, '', entry[0])
    cleanedTrainData.append((text, entry[1]))

In [7]:
# try print out a cleaned text
for i in range(0, 3):
    print(cleanedTrainData[i][0])

Greatly enjoyed this 1945 mystery thriller film about a young woman, Nina Foch,(Julia Ross) who is out of work and has fallen behind in her rent and is desperate to find work. Julia reads an ad in the local London newspaper looking for a secretary and rushes out to try and obtain this position. Julia obtains the position and is hired by a Mrs. Hughes, (Dame May Witty) who requires that she lives with her employer in her home and wants her to have no involvement with men friends and Julia tells them she has no family and is free to devote her entire time to this job. George Macready, (Ralph Hughes) is the son of Mrs. Hughes and has some very strange desires for playing around with knives. This was a low budget film and most of the scenes were close ups in order to avoid the expense of a background and costs for scenery. This strange family all live in a huge mansion off the Cornwall Coast of England and there is secret doors and plenty of suspense.
When this movie first came out back in

In [8]:
from transformers import BertTokenizer

2023-02-27 22:57:47.166878: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-27 22:57:47.345234: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-02-27 22:57:47.385659: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-02-27 22:57:48.066906: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; 

In [38]:
from transformers import AutoTokenizer

In [44]:
# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
sequence = "Using a Transformer network is simple"
tokens = tokenizer.tokenize(sequence)

print(tokens)
ids = tokenizer.convert_tokens_to_ids(tokens)

print(ids)

['using', 'a', 'transform', '##er', 'network', 'is', 'simple']
[2478, 1037, 10938, 2121, 2897, 2003, 3722]


In [ ]:
# Add special required formatting for BERT
# maxSentenceLen = 0
# for text, label in cleanedTrainData:
#     input_ids = tokenizer.encode(text, add_special_tokens=True)
# 
#     maxSentenceLen = max(maxSentenceLen, len(input_ids))
# 
# print(f'The max sentences length is: {maxSentenceLen}')

# ! BERT only take 512 max

Token indices sequence length is longer than the specified maximum sequence length for this model (1458 > 512). Running this sequence through the model will result in indexing errors


The max sentences length is: 1458


In [ ]:
sent = 'I hate this. So much'
tokened = tokenizer(sent, padding='max_lenght', max_length=512, add_special_tokens=True)
done = 

In [59]:
# test truncation methods [cut at the end -> shown to give better result]
# ? https://stackoverflow.com/questions/58636587/how-to-use-bert-for-long-text-classification?noredirect=1&lq=1
# ? https://github.com/huggingface/transformers/issues/4476#issuecomment-951445067
def encode_right_truncated(tokenizer, text, padding='max_length', max_length=512, add_special_tokens=True):
    out = tokenizer(text, padding=padding, max_length=max_length, add_special_tokens=add_special_tokens)
    tokenized = out['input_ids']
    print(f"Length of the inputID: {len(tokenized)}, length of token type: {len(out['token_type_ids'])}, length of mask: {len(out['attention_mask'])}")
    print(f"type: {out['token_type_ids']}")
    print(f"type: {out['attention_mask']}")
    if not add_special_tokens:
        truncated = tokenized[-max_length:]
    else:
        truncated = tokenized[0:1] + tokenized[-(max_length-1):]    # keep special start and end symbol
    
    return truncated

In [60]:
# Keeping only 512 word at the end
count = 0

for text, label in cleanedTrainData:
    input_ids = encode_right_truncated(tokenizer, text)
    print(input_ids)
    break
# print(f'The max sentences length is: {maxSentenceLen}')

Length of the inputID: 512, length of token type: 512, length of mask: 512
type: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

# Create input tensor

In [ ]:
# unpacking
wow = {
    'a': 1,
    'b': 20
}

def func (a, b):
    print(a, b)

# func(**{'a': 1, 'b': 2}, **{'c':4, 'd':5})

In [79]:
# LOOKING GOOD, now we can reuse the previous loop to create the input tensor

def encode_right_truncated(tokenizer, text, padding='max_length', max_length=512, add_special_tokens=True):
    out = tokenizer(text, padding=padding, max_length=max_length, add_special_tokens=add_special_tokens)
    tokenized = out['input_ids']
    tokenType = out['token_type_ids']
    attention = out['attention_mask']

    if not add_special_tokens:
        truncated = {
            'input_ids': tokenized[-max_length:],
            'token_type_ids': tokenType[-max_length:],
            'attention_mask': attention[-max_length:],
        }
    else:
        truncated = {
            'input_ids': tokenized[0:1] + tokenized[-(max_length-1):],    # keep special start and end symbol
            'token_type_ids': tokenType[0:1] + tokenType[-(max_length-1):],
            'attention_mask': attention[0:1] + attention[-(max_length-1):],
        }
    
    return truncated

In [84]:
from torch.utils.data import TensorDataset

testText = cleanedTrainData[0][0]
test = encode_right_truncated(tokenizer, testText)

inputIDTensor = torch.tensor(test['input_ids'])
attentionTensor = torch.tensor(test['attention_mask'])
labelTensor = torch.tensor(cleanedTrainData[0][1])

print(f'shape of inputID: {inputIDTensor.shape} and shape of attention: {attentionTensor.shape} and label shape: {labelTensor.shape}')

shape of inputID: torch.Size([512]) and shape of attention: torch.Size([512]) and label shape: torch.Size([])


Input of RNN should be tensor of (batch size, max length for the whole text, dimesion of the vocab vector [the size of the vector use to represent the vocab]) 
### in Bert, the shape is (batch, 512, 768) -> 768 is the size of each vocab vector in BERT